# Set Up Environment

## Create a virtual python environment to use as the kernel (using VS code)

Run the following in the terminal (for Windows):
- python -m venv .venv
- .venv\Scripts\activate
- python.exe -m pip install --upgrade pip
- pip install -r requirements.txt
- deactivate (to exit the venv)

If uisng VSCode, you should now be able to selct the .venv as the kernel to run the python notebook in.

![alt text](images/RaceEngChallenge/image.png)


## Set the path and import required dependencies 

In [8]:
import logging
import nest_asyncio

logging.basicConfig(level=logging.INFO)
nest_asyncio.apply()

import os
import sys

# Get the directory of the current script
script_dir = os.path.dirname(os.getcwd())

# Define the relative path (e.g., parent directory's src)
relative_path = os.path.join('..', 'src')

# Convert the relative path to an absolute path
absolute_path = os.path.abspath(os.path.join(script_dir, relative_path))

# Add the absolute path to sys.path
sys.path.insert(0, absolute_path)

from src.RaceEngChallenge import *

## Authenticate

Run this to authenticate yourself with the canopy sims API. There will be a keyboard entry prompt displayed. 

![alt text](images/RaceEngChallenge/image-1.png)

![alt text](images/RaceEngChallenge/image-2.png)

![alt text](images/RaceEngChallenge/image-3.png)

![alt text](images/RaceEngChallenge/image-4.png)

If the credentials are correct and authentication is successful, you will see:

![alt text](images/RaceEngChallenge/image-5.png)


In [2]:
session = await authenticate_canopy_sims_api(canopy.prompt_for_authentication())
user_id = session.authentication.user_id

Authenticated successfully!


# Start of script

## Inputs

In [3]:
# Enter sim_version
sim_version = '1.12300'
# Put your worksheet ID here. This canbe fouhnd in the URL of the worksheet: https://portal.canopysimulations.com/worksheets/client_id/worksheet_id
worksheet_id = '009d393737b04d2c8ab69553ee50e102'
# Enter the worksheet row names from which we extract the car configurations from. These rows will not be modified.
source_row_names = [
    'base'
]

## Run this to reset the worksheet

The following block can be called if you need to clean all the rows of the worksheet, except for those specified in the row_names list.

In [ ]:
#  First, reset the worksheet to only contain the rows we want to modify
reset_worksheet_result = await reset_worksheet(
    session=session,
    worksheet_id=worksheet_id,
    row_names=source_row_names
)

## Setup explorations

### Swept parameters and scalar limits

For this challenge, the objective is to identify the fastest legal car by varying certian parameters listed below, given a set of constraints. We will start with some fundamental parameters to be swept first.

The scalar limits must be adhered to by the identified car, else, the car is deemed to be illegal. Note that tLapTotal is also specified as a scalar limit with a max value of the lap time in the baseline configuration.. This is because it's quite a convenient way to introduce it into the function so that any cars slower than the baseline car are rejected.

Run the cell below to setup these parameters and limits.

In [5]:
swept_param_paths = [
    'car.chassis.hRideFSetup',
    'car.chassis.hRideRSetup',
    'car.aero.flapAngles.aFlapF',
    'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    'car.suspension.front.internal.torsionBar.kTorsionBar',
    'car.suspension.rear.internal.triSpring.kSpring',
    'car.chassis.carRunningMass.rWeightBalF',
    'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
    'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
    # 'car.powertrain.rearAxleTransmission.diff.MDiffDemandOutputs[0]',
    # 'car.powertrain.rearAxleTransmission.diff.MDiffDemandOutputs[2]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[0]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[1]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[2]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[3]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[4]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[5]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[6]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[7]',
    # 'car.control.brakeBalanceOptimisation.mapOutput.values[8]',
]

swept_parameter_data = [
    { 
        'id': 'hRideFSetup',
        'path': 'car.chassis.hRideFSetup',
        'min': 0.01,
        'max': 0.05
    },
    {
        'id': 'hRideRSetup',
        'path': 'car.chassis.hRideRSetup',
        'min': 0.01,
        'max': 0.08
    },
    {
        'id': 'aFlapF',
        'path': 'car.aero.flapAngles.aFlapF',
        'min': 0.4,
        'max': 1.0
    },
    {
        'id':'rWeightBalF',
        'path':'car.chassis.carRunningMass.rWeightBalF',
        'min': 0.45,
        'max': 0.47
    },
    {
        'id':'aCamberSetupF',
        'path':'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.0
    },
    {
        'id':'aCamberSetupR',
        'path':'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.0
    },
    # {
    #     'id': 'kAntiRollBarF',
    #     'path': 'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kAntiRollBarR',
    #     'path': 'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kTorsionBarF',
    #     'path': 'car.suspension.front.internal.torsionBar.kTorsionBar',
    #     'min': 0.0,
    #     'max': 10000.0
    # },
    # {
    #     'id': 'kTorsionBarR',
    #     'path': 'car.suspension.rear.internal.triSpring.kSpring',
    #     'min': 0.0,
    #     'max': 10000.0
    # }
]


dynamic_lap_scalar_limits = [
    {
        'id':'RaceEng_EBottoming',
        'max':0.5
    },
    {
        'id':'RaceEng_UndersteerT10Entry',
        'min':-4e-3,
    },
    {
        'id':'RaceEng_UndersteerT9',
        'min':-1.5e-3,
    },
    {
        'id':'RaceEng_aCamberFEOS',
        'min':-3.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_aCamberREOS',
        'min':-1.5,
        'max': 0.0
    },
    {
        'id':'RaceEng_kHeaveF',
        'max':850000
    },
    {
        'id':'RaceEng_kHeaveR',
        'max':400000
    },
    {
        'id':'RaceEng_kRoll',
        'max':350000
    },
    {
        'id':'RaceEng_rBrakeBalHighPressure',
        'min':0.6
    },
    {
        'id':'tLapTotal',
        'min':0.0,
        'max':77.2468
    }
]



## Check the scalar limits of the baseline study

Now, check the baseline study in the worksheet by retrieving the job scalar data and identify if the baseline car is legal. As seen, the baseline car is legal.

In [7]:
study_id = '5ff167be3eac448cb83389c3a7d1ff15' # base study id

scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id,
    sim_type='DynamicLap',
    scalar_limits=dynamic_lap_scalar_limits
)

# print(scalar_data)

disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=scalar_data
)

print(f"Number of disqualified jobs: {len(disqualified_jobs_data)}")

INFO:canopy.load_study_job:Loading job index 0


Number of disqualified jobs: 0


## Run new study with a desired exploration

Let's try a minimal heurisitcs incorporated approach to identify a faster car. One of the best ways to achieve this is to run some monte carlo explorations, whch we can setup and run the worksheet quite easily, following an approach shown in the cell below.

In [ ]:
# Create a Monte Carlo exploration config
exploration_config_id = await create_monte_carlo_exploration_config_from_swept_parameter_data(
    session=session,
    name='exploration_mc_0',
    sim_version=sim_version,
    swept_parameter_data=swept_parameter_data,
    n_monte_carlo_points=1000
)

# print(exploration_config_id)

result = await run_worksheet_row_study_with_configs_having_ids(
    session=session,
    worksheet_id=worksheet_id,
    source_row_name='base',
    sim_version=sim_version,
    row_suffix='exploration_mc_0',
    config_ids=[exploration_config_id]
)

worksheet_row_name = result['worksheet_row_name']
study_id = result['study_id']
worksheet_id = result['worksheet_id']

## Check if study with id is completed

Run the cell below id identify if all the jobs in the study have completed. If completed, proceed to the next step.

In [11]:
# Comment this out if you run the cell above
study_id = '6f11e70e7f7745069d9e80f4e1b4435b'

study = await canopy.load_study(
    session=session,
    study_id=study_id,
)

print(f"study_state: {study.document.data['studyState']}")

study_state: completed


## Re-check the scalar limits

The cell below loads the scalar data of all the jobs in the study and can take some time, especially for a Monte Carlo exploration with 100s of sims.

The warnings that are displayed are caused by jobs which have failed.

![alt text](images/RaceEngChallenge/image-6.png)

![alt text](images/RaceEngChallenge/image-7.png)

In [15]:
scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id,
    sim_type='DynamicLap',
    scalar_limits=dynamic_lap_scalar_limits
)

# print(scalar_data)

INFO:canopy.load_study_job:Loading job index 0
INFO:canopy.load_study_job:Loading job index 1
INFO:canopy.load_study_job:Loading job index 2
INFO:canopy.load_study_job:Loading job index 3
INFO:canopy.load_study_job:Loading job index 4
INFO:canopy.load_study_job:Loading job index 5
INFO:canopy.load_study_job:Loading job index 6
INFO:canopy.load_study_job:Loading job index 7
INFO:canopy.load_study_job:Loading job index 8
INFO:canopy.load_study_job:Loading job index 9
INFO:canopy.load_study_job:Loading job index 10
INFO:canopy.load_study_job:Loading job index 11
INFO:canopy.load_study_job:Loading job index 12
INFO:canopy.load_study_job:Loading job index 13
INFO:canopy.load_study_job:Loading job index 14
INFO:canopy.load_study_job:Loading job index 15
INFO:canopy.load_study_job:Loading job index 16
INFO:canopy.load_study_job:Loading job index 17
INFO:canopy.load_study_job:Loading job index 18
INFO:canopy.load_study_job:Loading job index 19
INFO:canopy.load_study_job:Loading job index 20
IN

## Check the disqualified jobs

Run the cell below to get the disqualified jobs which will be needed to identify the fastest car from the qualified jobs in the next step.

In [18]:
disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=scalar_data
)

print(disqualified_jobs_data)

#Get the number of disqualified jobs
disqualified_jobs_count = len(disqualified_jobs_data.keys())
print(f"Number of disqualified jobs: {disqualified_jobs_count}")

{'6f11e70e7f7745069d9e80f4e1b4435b-0': [{'scalar': 'RaceEng_EBottoming', 'scalar_value': 6.208225, 'min_limit': -inf, 'max_limit': 0.5}, {'scalar': 'RaceEng_UndersteerT10Entry', 'scalar_value': -0.011549, 'min_limit': -0.004, 'max_limit': inf}, {'scalar': 'RaceEng_UndersteerT9', 'scalar_value': -0.009122, 'min_limit': -0.0015, 'max_limit': inf}, {'scalar': 'RaceEng_aCamberREOS', 'scalar_value': -4.366567, 'min_limit': -1.5, 'max_limit': 0.0}, {'scalar': 'tLapTotal', 'scalar_value': 80.388272, 'min_limit': 0.0, 'max_limit': 77.2468}], '6f11e70e7f7745069d9e80f4e1b4435b-1': [{'scalar': 'RaceEng_UndersteerT10Entry', 'scalar_value': -0.011872, 'min_limit': -0.004, 'max_limit': inf}, {'scalar': 'RaceEng_UndersteerT9', 'scalar_value': -0.007278, 'min_limit': -0.0015, 'max_limit': inf}, {'scalar': 'RaceEng_aCamberFEOS', 'scalar_value': 0.101286, 'min_limit': -3.5, 'max_limit': 0.0}, {'scalar': 'RaceEng_aCamberREOS', 'scalar_value': -6.217797, 'min_limit': -1.5, 'max_limit': 0.0}], '6f11e70e7f7

## Get the job with the minimum tLapTotal

Run the cell below to idnetify the fastest car from the lot, if there is any. Oops, looks like there isn't any, so we will need to repeat the process with another monte carlo sweep with some adjusted ranges.

![alt text](images/RaceEngChallenge/image-8.png)

In [19]:
# Create a list of job_ids to exclude from the analysis from the keys of disqualified_jobs_data
excluded_job_ids = list(disqualified_jobs_data.keys())

results = await get_job_with_minimum_scalar_value(
    session=session,
    study_id=study_id,
    sim_type='DynamicLap',
    scalar_name='tLapTotal',
    excluded_job_ids=excluded_job_ids
)

print(results)

INFO:canopy.load_study_job:Loading job index 0
INFO:canopy.load_study_job:Loading job index 1
INFO:canopy.load_study_job:Loading job index 2
INFO:canopy.load_study_job:Loading job index 3
INFO:canopy.load_study_job:Loading job index 4
INFO:canopy.load_study_job:Loading job index 5
INFO:canopy.load_study_job:Loading job index 6
INFO:canopy.load_study_job:Loading job index 7
INFO:canopy.load_study_job:Loading job index 8
INFO:canopy.load_study_job:Loading job index 9
INFO:canopy.load_study_job:Loading job index 10
INFO:canopy.load_study_job:Loading job index 11
INFO:canopy.load_study_job:Loading job index 12
INFO:canopy.load_study_job:Loading job index 13
INFO:canopy.load_study_job:Loading job index 14
INFO:canopy.load_study_job:Loading job index 15
INFO:canopy.load_study_job:Loading job index 16
INFO:canopy.load_study_job:Loading job index 17
INFO:canopy.load_study_job:Loading job index 18
INFO:canopy.load_study_job:Loading job index 19
INFO:canopy.load_study_job:Loading job index 20
IN

Checking job 6f11e70e7f7745069d9e80f4e1b4435b-4 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-5 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-13 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-24 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-36 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-37 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-38 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-44 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-78 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-116 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-180 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-191 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-192 for scalar tLapTotal...
Checking job 6f11e70e7f7745069d9e80f4e1b4435b-194

## No jobs meeting the requirements found, so run again.

Now, the question of how to identify which numbers to use for the next iteration comes up. 

If you want to fully automate the process, this should be possible by writing up a fancy algorithm of your choosing and calling these functions iteratively until you hit a limit.

Or, you can go quick and dirty by taking a look at the cross section plots and visually observing the trend of tLapTotal:DynamicLap with each swept parameter.

![alt text](images/RaceEngChallenge/image-9.png)

In [20]:
swept_parameter_data_1 = [
    { 
        'id': 'hRideFSetup',
        'path': 'car.chassis.hRideFSetup',
        'min': 0.02,
        'max': 0.03
    },
    {
        'id': 'hRideRSetup',
        'path': 'car.chassis.hRideRSetup',
        'min': 0.06,
        'max': 0.08
    },
    {
        'id': 'aFlapF',
        'path': 'car.aero.flapAngles.aFlapF',
        'min': 0.4,
        'max': 0.6
    },
    # {
    #     'id':'rWeightBalF',
    #     'path':'car.chassis.carRunningMass.rWeightBalF',
    #     'min': 0.45,
    #     'max': 0.45
    # },
    {
        'id':'aCamberSetupF',
        'path':'car.suspension.front.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.1
    },
    {
        'id':'aCamberSetupR',
        'path':'car.suspension.rear.external.aCamberSetupAlignment.aCamberSetup',
        'min': -0.1,
        'max': 0.1
    },
    # {
    #     'id': 'kAntiRollBarF',
    #     'path': 'car.suspension.front.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kAntiRollBarR',
    #     'path': 'car.suspension.rear.internal.antiRollBar.kAntiRollBar',
    #     'min': 0.0,
    #     'max': 100000.0
    # },
    # {
    #     'id': 'kTorsionBarF',
    #     'path': 'car.suspension.front.internal.torsionBar.kTorsionBar',
    #     'min': 0.0,
    #     'max': 10000.0
    # },
    # {
    #     'id': 'kTorsionBarR',
    #     'path': 'car.suspension.rear.internal.triSpring.kSpring',
    #     'min': 0.0,
    #     'max': 10000.0
    # }
]

## Run study with new exploration

Let's call this exploration_mc_1

In [ ]:
# List to hold the new config ids
config_ids = [
]

# Create a Monte Carlo exploration config
exploration_config_id = await create_monte_carlo_exploration_config_from_swept_parameter_data(
    session=session,
    name='exploration_mc_1',
    sim_version=sim_version,
    swept_parameter_data=swept_parameter_data_1,
    n_monte_carlo_points=1000
)

# print(exploration_config_id)

result = await run_worksheet_row_study_with_configs_having_ids(
    session=session,
    worksheet_id=worksheet_id,
    source_row_name='base',
    sim_version=sim_version,
    row_suffix='exploration_mc_1',
    config_ids=[exploration_config_id]
)

worksheet_row_name = result['worksheet_row_name']
study_id = result['study_id']
worksheet_id = result['worksheet_id']

# Check if study with id is completed

In [21]:
# Comment this out if you run the cell above
study_id = '48fda98fb8a44663b831df56272bb99f'

study = await canopy.load_study(
    session=session,
    study_id=study_id,
)

print(f"study_state: {study.document.data['studyState']}")

study_state: completed


## Re-check the scalar limits

In [22]:
scalar_data = await get_scalar_data_from_jobs_in_study_with_id(
    session=session,
    study_id=study_id,
    sim_type='DynamicLap',
    scalar_limits=dynamic_lap_scalar_limits
)

# print(scalar_data)

INFO:canopy.load_study_job:Loading job index 0
INFO:canopy.load_study_job:Loading job index 1
INFO:canopy.load_study_job:Loading job index 2
INFO:canopy.load_study_job:Loading job index 3
INFO:canopy.load_study_job:Loading job index 4
INFO:canopy.load_study_job:Loading job index 5
INFO:canopy.load_study_job:Loading job index 6
INFO:canopy.load_study_job:Loading job index 7
INFO:canopy.load_study_job:Loading job index 8
INFO:canopy.load_study_job:Loading job index 9
INFO:canopy.load_study_job:Loading job index 10
INFO:canopy.load_study_job:Loading job index 11
INFO:canopy.load_study_job:Loading job index 12
INFO:canopy.load_study_job:Loading job index 13
INFO:canopy.load_study_job:Loading job index 14
INFO:canopy.load_study_job:Loading job index 15
INFO:canopy.load_study_job:Loading job index 16
INFO:canopy.load_study_job:Loading job index 17
INFO:canopy.load_study_job:Loading job index 18
INFO:canopy.load_study_job:Loading job index 19
INFO:canopy.load_study_job:Loading job index 20
IN

## Check the disqualified jobs

In [23]:
disqualified_jobs_data = get_disqualified_jobs_data_in_study_with_id_due_to_scalar_limit_violations(
    scalar_data=scalar_data
)

print(disqualified_jobs_data)

#Get the number of disqualified jobs
disqualified_jobs_count = len(disqualified_jobs_data.keys())
print(f"Number of disqualified jobs: {disqualified_jobs_count}")

{'48fda98fb8a44663b831df56272bb99f-1': [{'scalar': 'RaceEng_EBottoming', 'scalar_value': 0.716151, 'min_limit': -inf, 'max_limit': 0.5}, {'scalar': 'RaceEng_UndersteerT9', 'scalar_value': -0.002462, 'min_limit': -0.0015, 'max_limit': inf}], '48fda98fb8a44663b831df56272bb99f-2': [{'scalar': 'RaceEng_EBottoming', 'scalar_value': 1.228417, 'min_limit': -inf, 'max_limit': 0.5}, {'scalar': 'RaceEng_UndersteerT10Entry', 'scalar_value': -0.010127, 'min_limit': -0.004, 'max_limit': inf}, {'scalar': 'RaceEng_UndersteerT9', 'scalar_value': -0.004248, 'min_limit': -0.0015, 'max_limit': inf}, {'scalar': 'RaceEng_aCamberREOS', 'scalar_value': -1.852298, 'min_limit': -1.5, 'max_limit': 0.0}], '48fda98fb8a44663b831df56272bb99f-3': [{'scalar': 'RaceEng_EBottoming', 'scalar_value': 1.464386, 'min_limit': -inf, 'max_limit': 0.5}, {'scalar': 'RaceEng_aCamberREOS', 'scalar_value': 0.289176, 'min_limit': -1.5, 'max_limit': 0.0}], '48fda98fb8a44663b831df56272bb99f-4': [{'scalar': 'RaceEng_UndersteerT10Entry

## Get the job with the minimum tLapTotal

In [24]:
# Create a list of job_ids to exclude from the analysis from the keys of disqualified_jobs_data
excluded_job_ids = list(disqualified_jobs_data.keys())

results = await get_job_with_minimum_scalar_value(
    session=session,
    study_id=study_id,
    sim_type='DynamicLap',
    scalar_name='tLapTotal',
    excluded_job_ids=excluded_job_ids
)

print(results)

INFO:canopy.load_study_job:Loading job index 0
INFO:canopy.load_study_job:Loading job index 1
INFO:canopy.load_study_job:Loading job index 2
INFO:canopy.load_study_job:Loading job index 3
INFO:canopy.load_study_job:Loading job index 4
INFO:canopy.load_study_job:Loading job index 5
INFO:canopy.load_study_job:Loading job index 6
INFO:canopy.load_study_job:Loading job index 7
INFO:canopy.load_study_job:Loading job index 8
INFO:canopy.load_study_job:Loading job index 9
INFO:canopy.load_study_job:Loading job index 10
INFO:canopy.load_study_job:Loading job index 11
INFO:canopy.load_study_job:Loading job index 12
INFO:canopy.load_study_job:Loading job index 13
INFO:canopy.load_study_job:Loading job index 14
INFO:canopy.load_study_job:Loading job index 15
INFO:canopy.load_study_job:Loading job index 16
INFO:canopy.load_study_job:Loading job index 17
INFO:canopy.load_study_job:Loading job index 18
INFO:canopy.load_study_job:Loading job index 19
INFO:canopy.load_study_job:Loading job index 20
IN

Checking job 48fda98fb8a44663b831df56272bb99f-0 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-23 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-64 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-85 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-106 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-107 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-121 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-159 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-169 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-174 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-208 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-214 for scalar tLapTotal...
Checking job 48fda98fb8a44663b831df56272bb99f-226 for scalar tLapTotal...
Job ID: 48fda98fb8a44663b831df56272bb99f-22

# Found one!

Woop woop, looks like I found a faster 'legal' car here!

![alt text](images/RaceEngChallenge/image-10.png)

And that's it, hope this tutorial was helpful to demonstrate this iterative process.